In [ ]:
import sys
import datetime as dt
from datetime import timedelta
import json
from pathlib import Path
import report_utils as ru
from jinja2 import Environment, FileSystemLoader, DebugUndefined
import csv
from pprint import pp
from translation_utils import TranslatorConfig

sys.version_info

In [ ]:
report_json = "unset"
html_out_folder = "unset"
logical_date = "unset"
incident_date = "unset"
shared_data_dir = "unset"
dagrun_data_dir = "unset"
static_dir = "unset"
template_dir = "templates"
crisis_name = "unset"
update_freq = "unset"
key_obs_dir = "unset"
country = "unset"

In [ ]:
execution_date_str = logical_date
logical_date = dt.date.fromisoformat(logical_date)
incident_date = dt.date.fromisoformat(incident_date)
report_json = Path(dagrun_data_dir) / report_json
html_out_folder = Path(shared_data_dir) / html_out_folder
template_path = Path(static_dir) / template_dir

In [ ]:
html_out_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
translator_config = TranslatorConfig(
    translation_folder=template_path / "active_crisis_report" / "translations",
)

In [ ]:
class MissingDataError(Exception):
    pass


def get_update_num():
    return 1


def get_manual_key_obs():
    try:
        return (Path(key_obs_dir) / (execution_date_str + ".html")).read_text()
    except FileNotFoundError:
        return "No key observations written for this report."


def get_last_report_date():
    return dt.date(1990, 1, 1)


def get_last_stable_date():
    return dt.date(1990, 1, 1)


def get_last_stable_report_num():
    return 1


def get_last_prep_report_num():
    return 1


def stats_box_csv_to_values(csv_path):
    with open(csv_path, newline="") as csvfile:
        reader = csv.DictReader(csvfile)

        def _conditional_round(value):
            try:
                return round(float(value))
            except ValueError:
                return value

        return {key: _conditional_round(value) for key, value in next(reader).items()}

In [ ]:
page_2_required_field_list = [
    "top3_adm_hosts",
    "residents_hosts_vs_aa",
    "excess_arrivals_excess_residents_table",
    "all_excess_arrivals_zoomed",
    "displaced_stats_boxes",
]

print(report_json)
print(html_out_folder)

jinja_env = Environment(
    loader=FileSystemLoader(template_path), undefined=DebugUndefined
)

date_range_end = ru.ReportDate(logical_date.year, logical_date.month, logical_date.day)
date_range_start = ru.ReportDate(
    incident_date.year, incident_date.month, incident_date.day
)

report_data = json.loads(report_json.read_text())
report_data

In [ ]:
good_regions = {
    affected_area: aa_data
    for affected_area, aa_data in report_data["affected_areas"].items()
    if all(field in aa_data.keys() for field in page_2_required_field_list)
}
if len(good_regions) == 0:
    raise MissingDataError(
        f"No areas with data detected.\nAffected area data:{report_json}"
    )
good_regions

In [ ]:
national_data = report_data["national"]
crisis_general_args = dict(
    name_of_crisis=crisis_name,
    pub_date=date_range_end,
    update_freq_days=update_freq,
    total_pages=2 if len(good_regions) == 1 else len(good_regions) + 2,
    update_num=get_update_num(),
    period=ru.ReportPeriod(date_range_start, date_range_end),
    period_short=ru.ReportPeriod(date_range_start, date_range_end).short(),
    crisis_date=date_range_end,  # Check this
    country=country,
)

In [ ]:
stats_boxes = stats_box_csv_to_values(national_data["displaced_stats_boxes"])

In [ ]:
running_page_num = 2
if len(report_data["affected_areas"]) > 1:
    cover = ru.CrisisTemplateCover(
        manual_key_observations=get_manual_key_obs(),
        displaced_subs_table=ru.ReportTable(
            national_data["excess_arrivals_excess_residents_table"], rows=10
        ),
        location_displacement_table=ru.ReportTable(
            national_data["excess_arrivals_excess_residents_table"], rows=10
        ),
        area_map=ru.ReportImage(national_data["all_excess_arrivals"]),
        # TODO: Check with Rob that these are correct
        remaining_displaced=stats_boxes["total_displaced_in_known_areas"],
        displaced_neighbourhoods=stats_boxes["total_displaced_from_aa"],
        newly_displaced=stats_boxes["newly_displaced"],
        displaced_plot=ru.ReportImage(
            national_data["excess_arrivals_excess_residents_barchart"]
        ),
        newly_displaced_ts=ru.ReportImage(national_data["top3_adm_hosts_multi_aa"]),
        page_num=1,
        **crisis_general_args,
    )
    for affected_area, aa_data in good_regions.items():
        # try:
        region_stats_boxes = stats_box_csv_to_values(aa_data["displaced_stats_boxes"])
        region = ru.CrisisTemplateRegion(
            affected_area=affected_area,
            total_displaced=region_stats_boxes["value"],
            displaced_since_last="TODO",  # region_stats_boxes["defecit"],
            area_map=ru.ReportImage(aa_data["all_excess_arrivals_zoomed"]),
            newly_displaced_ts=ru.ReportImage(aa_data["top3_adm_hosts"]),
            remaining_displaced_ts=ru.ReportImage(aa_data["residents_hosts_vs_aa"]),
            page_num=running_page_num,
            **crisis_general_args,
        )
        running_page_num += 1
        print(region)

        ru.render_report(
            region,
            template_name="active_crisis_report/affected_area.html",
            style="active_crisis_report/active_crisis.css",
            env=jinja_env,
            out_folder=html_out_folder,
            out_name=f"{running_page_num:02}_{region.affected_area}.html",
            translator_config=translator_config,
        )
else:  # Only one AA for this branch
    aa_data = next(iter(good_regions.values()))
    aa_map = aa_data["all_excess_arrivals_zoomed"]
    cover = ru.CrisisTemplateCover(
        manual_key_observations=get_manual_key_obs(),
        displaced_subs_table=ru.ReportTable(
            national_data["excess_arrivals_excess_residents_table"], rows=10
        ),
        location_displacement_table=ru.ReportTable(
            national_data["excess_arrivals_excess_residents_table"], rows=10
        ),
        area_map=ru.ReportImage(aa_map),
        # TODO: Check with Rob that these are correct
        remaining_displaced=stats_boxes["total_displaced_in_known_areas"],
        displaced_neighbourhoods=stats_boxes["total_displaced_from_aa"],
        newly_displaced=stats_boxes["newly_displaced"],
        displaced_plot=ru.ReportImage(
            national_data["excess_arrivals_excess_residents_barchart"]
        ),
        newly_displaced_ts=ru.ReportImage(national_data["top3_adm_hosts_multi_aa"]),
        page_num=1,
        **crisis_general_args,
    )

In [ ]:
ru.render_report(
    cover,
    template_name="active_crisis_report/cover.html",
    style="active_crisis_report/active_crisis.css",
    env=jinja_env,
    out_folder=html_out_folder,
    out_name="01_cover.html",
    translator_config=translator_config,
)

In [ ]:
last_report_date = get_last_report_date()
last_stable_date = get_last_stable_date()
back_matter = ru.CrisisTemplateBackMatter(
    month=logical_date.month,
    year=logical_date.year,
    prep_month=last_report_date.month,
    prep_year=last_report_date.year,
    prep_num=get_last_prep_report_num(),
    strec_month=last_stable_date.month,
    strec_year=last_stable_date.year,
    strec_num=get_last_stable_report_num(),
    page_num=running_page_num,
    **crisis_general_args,
)
back_matter

In [ ]:
ru.render_report(
    back_matter,
    template_name="active_crisis_report/back_matter.html",
    style="active_crisis_report/active_crisis.css",
    env=jinja_env,
    out_folder=html_out_folder,
    out_name=f"{running_page_num:02}_back_matter.html",
    translator_config=translator_config,
)